#Crawl Data

In [ ]:
#Import các thư viện cần thiết
import concurrent.futures
import requests
import csv
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup

#Định nghĩa hàm fetch_links(page_number) để lọc ra các link bán nhà từ trang web bất động sản và đưa vào output_list
def fetch_links(page_number):
    url = 'https://batdongsan.vn/ban-nha-quan-6-ho-chi-minh/p'+ str(page_number)
    #Sử dụng thư viện request để lấy html từ từng trang
    response = requests.get(url)
    output_list = []
    if response.status_code == 200:#Câu điều kiện check xem phản hồi từ web có bị lỗi hay không
        soup = BeautifulSoup(response.text, 'html.parser')# Sử dụng BeautifulSoup để phân tích cú pháp html của trang web
        product_links = soup.find_all(class_="image cover") #  Lấy thông tin từ lớp image cover
        for link in product_links:
            href = link.find('a')['href']#Lấy nội dung của thẻ a href để lấy link của từng thông tin bán nhà đất
            output_list.append(href) # Đưa vào output_list bằng phương thức append
        print(f'Yêu cầu không thành công. Mã trạng thái: {response.status_code}')
    return output_list

#Sử dụng ThreadPoolExecutor để thực hiện đa luồng và lưu trữ kết quả vào results
with ThreadPoolExecutor(max_workers=20) as executor:
    pages = range(1, 22)
    results = list(executor.map(fetch_links, pages))

output_list = [url for sublist in results for url in sublist]



In [ ]:
print(output_list)
print(len(output_list))


['https://batdongsan.vn/ban-nha-tran-xuan-soan-phuong-tan-hung-q7-12m2-hem-xe-may-gia-1x-ty-r285107', 'https://batdongsan.vn/quan-7-thong-so-cuc-dep-4x13m-cach-1-can-ra-o-to-r284971', 'https://batdongsan.vn/can-ban-nha-nat-duong-tan-my-gan-truong-dai-hoc-gia-re-r284384', 'https://batdongsan.vn/nha-mat-pho-huynh-tan-phat-dtsd-84m2-nhinh-3-ty-r283984', 'https://batdongsan.vn/can-ban-gap-nha-mat-tien-duong-so-39-gia-re-chi-7-ty-850-r283574', 'https://batdongsan.vn/ban-nha-1lau-shr-hem-1283-huynh-tan-phat-dtsd-56m2-gia-35ty-quan-7-r283431', 'https://batdongsan.vn/ket-tien-can-ban-gap-nha-1-lau-mat-tien-duong-so-17-ngay-truong-nguyen-huu-tho-gia-chi-15-ty-r283415', 'https://batdongsan.vn/quan-7-oto-ngu-trong-nha-4x20m-duong-nhua-chi-nhinh-10ty-r283410', 'https://batdongsan.vn/ban-nha-mat-tien-nguyen-van-quy-quan-7-dtsd-84m2-nhinh-3ty-r282747', 'https://batdongsan.vn/hot-hot-chu-chin-di-xuat-ngoai-de-lai-nha-mat-tien-duong-so-tan-quy-r282267', 'https://batdongsan.vn/ban-nha-3-tang-st-hem-xe-

In [ ]:
# Định nghĩa hàm fetch_url(link) để lấy thông tin từ trang web bất động sản
def fetch_url(link):
    response = requests.get(link)
    attributes = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        div_content = soup.find('div', class_='param')
        if div_content:
            # Tìm các phần tử div có class_='param'
            for div in div_content:
                list_items = div_content.find_all('li')
                # Tìm tất cả các phần tử li trong div
                for item in list_items:
                    title_strong = item.find('strong')
                    if title_strong:
                        # Lấy nội dung trong thẻ strong làm tiêu đề
                        title = title_strong.text.strip().replace(":", "")
                        # Lấy nội dung trong phần tử li, loại bỏ tiêu đề và thực hiện lọc dữ liệu
                        value = item.text.replace(title_strong.text, '').strip()
                        if value and title:
                            attributes[title] = value
                        else:
                            attributes[title] = None
        else:
            print('Không có thông tin')
            return None

        # Lấy các thuộc tính cần thiết từ attributes
        area = attributes.get('Diện tích')
        bedrooms = attributes.get('Phòng ngủ')
        bathrooms = attributes.get('Phòng WC')
        address = attributes.get('Địa chỉ')

        price = soup.find('strong', class_='price').text.strip()
        if price:
            price_tag = price
        else:
            price_tag = None

        description_tag = soup.find('div', class_='content')
        if description_tag:
            description = description_tag.text.strip()
        else:
            description = None

        return [price_tag, area, bedrooms, bathrooms, address, description]
    else:
        print(f'Yêu cầu không thành công. Mã trạng thái: {response.status_code}')
        return None

# Mở file CSV để ghi dữ liệu lấy được từ trang web
with open('data_crawl.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Price','Area', 'Bedrooms', 'WC', 'Address','Describe'])

    # Sử dụng ThreadPoolExecutor để thực hiện đa luồng
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_url = {executor.submit(fetch_url, link): link for link in output_list}
        for future in concurrent.futures.as_completed(future_to_url):
            data = future.result()
            if data is not None:
                writer.writerow(data)